> This notebook contains the evolutionary dataset first analysis.

In [1]:
import pandas as pd
import numpy as np

import re
import csv

from Bio import AlignIO

from Bio.Align import AlignInfo

from Bio.Align import MultipleSeqAlignment
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

import os

### 1. Data Loading

> We have both xls and fasta format for the evolutionary dataset. 

In [2]:
# Data Paths
FASTA_msa_seq = 'data/msa/4.Fasta_all_7k.txt'
XLS_msa_seq = 'data/msa/Dataset_S4.xls'
XLS_msa_seq_csv = 'data/msa/Dataset_S4.csv'

# Read the Excel file and write to CSV
df = pd.read_excel(XLS_msa_seq)
df.to_csv(XLS_msa_seq_csv, index=False)

# Data Loading
#fasta_msa = pd.read_csv(FASTA_msa_seq,  delimiter='\t')
XLS_msa = pd.read_csv(XLS_msa_seq_csv)



In [3]:
# Replace 'your_file.csv' with the path to your CSV file
file_path = 'data/BLAT_ECOLX_Ranganathan2015-2500/data.csv'

# Read the CSV file
df = pd.read_csv(file_path)

# Assuming the second column is at index 1 since Python is zero-indexed
second_column = df.iloc[:, 1]

# Find the minimum and maximum values
min_value = second_column.min()
max_value = second_column.max()
number_of_lines = len(df.index)

print(f"The minimum value in the second column is: {min_value}")
print(f"The maximum value in the second column is: {max_value}")
print(f"The number of lines in the CSV file is: {number_of_lines}")

The minimum value in the second column is: -3.743276064
The maximum value in the second column is: 0.3557906625
The number of lines in the CSV file is: 4807


In [13]:
# Replace 'your_file.a2m' with the path to your .a2m file
file_path1 = 'alignments/Q2NB98.a2m'

# Initialize a counter
count = 0

# Open the file and count '>' instances
with open(file_path1, 'r') as file:
    for line in file:
        # Increment count if '>' is found at the beginning of a line
        if line.startswith('>'):
            count += 1

print(f"The number of '>' instances in the file is: {count}")

The number of '>' instances in the file is: 9255


In [5]:
display(XLS_msa)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,NaN,User note: GenBank Accession numbers marked wi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Database Source,Sequence ID,GenBank Accession,Primary Structure,GXNCRFLQ Motif,Protein Length,Domain Structure,Functional Cluster,Primary Effector,Primary Effector Gene Ontology,Linker Length,Number of Predicted Transmembrane Helices,Transmembrane Helix Topology,Kingdom,Phylum,Class,Order,Family,Genus,Species
3,Interpro,A0A009EVU7,EXA65901.1,MTSFYNGGTSQEIFNLIGQAAALEDIIDAITVWLGDKIPDALVSVM...,GKNCRILQ,850.0,"[N_terminus : (0, 0) ---> GAF_2 : (9, 166) ---...","['EAL', 'GAF', 'GGDEF', 'LOV', 'PAS']",GGDEF,Cyclic Nucleotide Biosythesis,12.0,0.0,NaN,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,baumannii
4,Interpro,A0A009JQP1,EXB31618.1,MNNNCDDHDTYHIFNLIGQHTELHLILSAASTWLESRISNVLVAIM...,GKTCHFLQ,855.0,"[N_terminus : (0, 0) ---> GAF_2 : (26, 166) --...","['EAL', 'GAF', 'GGDEF', 'LOV', 'PAS']",GGDEF,Cyclic Nucleotide Biosythesis,12.0,0.0,NaN,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,baumannii
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6780,OneKP,ZYAX_2075737,KU702188,MEPSNPGNTNNSFPSGNVVRKVLDDSPAIISPLPRDSRGSLEVFNP...,GKNCRFLQ,1083.0,"[N_terminus : (0, 0) ---> TLOV : (286, 663) --...","['Pkinase', 'TLOV']",Pkinase,Kinase Activity,83.0,0.0,NaN,Land Plants,Pinophyta,Pinopsida,Pinales,Taxaceae,Taxus,cuspidata
6781,OneKP,ZZOL_2003873,KU702189,MESTSGSKQDHYFSQRSSSMPKPVRDSLAQLPRDSRGSLEVFNPQG...,GRNCRFLQ,1008.0,"[N_terminus : (0, 0) ---> TLOV : (232, 597) --...","['Pkinase', 'TLOV']",Pkinase,Kinase Activity,83.0,0.0,NaN,Land Plants,Lycopodiophyta,Isoetopsida,Selaginellales,Selaginellaceae,Selaginella,lepidophylla
6782,OneKP,ZZOL_2007563,KU702190,MALQNGSLLSSDRKCVRSPRMKNHKLIVSQSLIRSYRASIQDELQK...,GRNCRFLQ,394.0,"[N_terminus : (0, 0) ---> TLOV : (49, 354) ---...",['TLOV'],Short_LOV,Short LOV,0.0,0.0,NaN,Land Plants,Lycopodiophyta,Isoetopsida,Selaginellales,Selaginellaceae,Selaginella,lepidophylla
6783,OneKP,ZZOL_2011001,KU702191#,MLVEVSQFTEGSKDKAVRPNGMPESLIKYDARQKDSVVSSVSELVE...,GKNCRFLQ,465.0,"[N_terminus : (0, 0) ---> LOV : (142, 247) ---...","['LOV', 'Pkinase']",Pkinase,Kinase Activity,83.0,0.0,NaN,Land Plants,Lycopodiophyta,Isoetopsida,Selaginellales,Selaginellaceae,Selaginella,lepidophylla


###  2. Convert MSA .a2m file to adequate .a2m format

> The alignment requirements are the folowing:
The first step of the EVmutation method is to compute a pairwise model of sequences from a family sequence alignment, e.g. the included example file (example/msa_comparison.a2m). This alignment has to be in aligned FASTA/A2M format and must fulfill the following requirements:
* The target sequence may not contain any gaps.
* Columns that should be excluded from model inference (e.g. too many gaps) must be represented by lowercase characters. Gaps in these positions must be represented by a dot (".") rather than a dash ("-").
* The identifier of the target sequence has to be passed to plmc with the -f parameter (see below)

In [6]:
from Bio import AlignIO

def calculate_sequence_homology(fasta_file):
    alignment = AlignIO.read(fasta_file, "fasta")
    num_sequences = len(alignment)
    length_of_alignment = alignment.get_alignment_length()
    homology_count = 0

    for i in range(length_of_alignment):
        column = alignment[:, i]
        if column.count(column[0]) == num_sequences:
            homology_count += 1

    homology_percentage = (homology_count / length_of_alignment) * 100
    return homology_percentage

In [7]:
def convert_to_correct_a2m(fasta_file, output_file):
    """
    Processes a protein multiple sequence alignment in .a2m format. Wherever there are lowercase
    letters in the WT sequence and dashes in the corresponding positions in the other
    sequences, converts the dashes to dots. If there is a lowercase letter in the
    corresponding position of the other sequences, it is left as is.

    Args:
    fasta_file (str): Path to the input .a2m file.
    output_file (str): Path to the output .a2m file.

    Raises:
    FileNotFoundError: If the input file is not found.
    """

    try:
        # Read the alignment from the .a2m file
        alignment = AlignIO.read(fasta_file, "fasta")
    except FileNotFoundError:
        raise FileNotFoundError(f"File {fasta_file} not found.")

    # Get the WT sequence (first sequence in the alignment)
    wt_sequence = str(alignment[0].seq)

    # Process each sequence in the alignment
    processed_alignment = MultipleSeqAlignment([])
    for record in alignment:
        sequence = str(record.seq)
        processed_seq = ""
        for i, char in enumerate(sequence):
            wt_char = wt_sequence[i]
            if wt_char.islower() and char == '-':
                processed_seq += '.'
            else:
                processed_seq += char
        processed_record = SeqRecord(Seq(processed_seq), id=record.id, description=record.description)
        processed_alignment.append(processed_record)

    # Save the processed alignment to the specified output file
    AlignIO.write(processed_alignment, output_file, "fasta")

> The initial .a2m format is generated using the hh-suite repo (which should be downloaded): https://github.com/soedinglab/hh-suite, which allows for conversion between different MSA file formats. The command that should be used is the following where the reformat function is uised: reformat.pl file_to_be_converted.a3m converted_file.a2m

In [21]:
def convert_to_correct_a2m(a2m_filename, output_filename):
    """
    Processes an A2M file using Biopython by:
    - Removing dashes and dots from the wild type sequence (the first sequence)
    - Removing corresponding positions in all other sequences
    - Renaming the wild type sequence to '>Q2NB98'

    :param a2m_filename: Path to the input A2M file.
    :param output_filename: Path for the output processed file.
    """
    # Read the alignment
    alignment = AlignIO.read(a2m_filename, "fasta")
    
    # Get the wild type sequence (first sequence in the alignment)
    wild_type_seq = str(alignment[0].seq)
    positions_to_keep = [i for i, char in enumerate(wild_type_seq) if char not in '-.']

    # Process and rename the wild type sequence
    processed_wild_type_seq = ''.join(wild_type_seq[i] for i in positions_to_keep)
    processed_wild_type_record = SeqRecord(Seq(processed_wild_type_seq), id="Q2NB98", description="")

    # Create a new alignment with processed sequences
    new_alignment = MultipleSeqAlignment([processed_wild_type_record])
    for record in alignment[1:]:
        new_seq = ''.join(str(record.seq)[i] for i in positions_to_keep if i < len(record.seq))
        new_record = SeqRecord(Seq(new_seq), id=record.id, description="")
        new_alignment.append(new_record)

    # Write the new alignment to a file
    AlignIO.write(new_alignment, output_filename, "fasta")

In [22]:
initial_a2m_format = 'alignments/LOV_Q2NB98.a2m'
correct_a2m_format = 'alignments/Q2NB98.a2m'
convert_to_correct_a2m(initial_a2m_format, correct_a2m_format)

>Let's apply this function to the fasta file (LOV domains starting with EL222 aligned.fasta).

In [8]:
homology = calculate_sequence_homology('alignments/Q2NB98.a2m')
print(f"Sequence Homology: {homology}%")

Sequence Homology: 0.0%


>Function to remove sequences that have more than 30% gaps.

In [9]:
def remove_high_gap_sequences(input_file, output_file, gap_threshold=30):
    """
    Removes sequences from an alignment file (.a2m) that have more than a certain
    percentage of gaps.

    Args:
    input_file (str): Path to the input .a2m file.
    output_file (str): Path to save the filtered .a2m file.
    gap_threshold (int): The threshold percentage of gaps (default 30%).
    """

     # Read the alignment
    alignment = AlignIO.read(input_file, "fasta")
    total_sequences = len(alignment)

    # Filter sequences with gap percentage less than the threshold
    filtered_sequences = [
        record for record in alignment if ((str(record.seq).count('-') + str(record.seq).count('.')) / len(record.seq)) * 100 < gap_threshold
    ]


    # Rename the first sequence
    filtered_sequences[0].id = 'Q2NB98_30perc'
    filtered_sequences[0].name = 'Q2NB98_30perc'
    filtered_sequences[0].description = ''  # Clearing the description


    # Create a new alignment with the filtered and renamed sequences
    filtered_alignment = MultipleSeqAlignment(filtered_sequences)

    # Write the filtered alignment to a file
    AlignIO.write(filtered_alignment, output_file, "fasta")

    # Calculate the number and percentage of removed sequences
    num_removed = total_sequences - len(filtered_sequences)
    perc_removed = (num_removed / total_sequences) * 100

    return num_removed, perc_removed

In [10]:
LOV_msa_a2m_30gaps_removed = 'alignments/Q2NB98_30perc.a2m'
LOV_msa_a2m_dashes = 'alignments/Q2NB98.a2m'
num_removed, perc_removed = remove_high_gap_sequences(LOV_msa_a2m_dashes, LOV_msa_a2m_30gaps_removed, gap_threshold=30)

print(f"Number of sequences removed: {num_removed}")
print(f"Percentage of sequences removed: {perc_removed:.2f}%")

Number of sequences removed: 4363
Percentage of sequences removed: 47.14%


###  3. Domain of interest range analysis

In [11]:
def extract_lov_domain_tuples(df):
    lov_domain_data = []

    for index, row in df.iterrows():
        # Skip initial rows that don't contain the relevant data
        if index < 3:  # Assuming data starts from the fifth row (index 3)
            continue

        # Access columns by position using .iloc
        protein_id = row.iloc[1]  # Assuming the protein ID is in the second column
        domain_info = row.iloc[6]  # Assuming the domain info is in the sixth column

        print(f"Debug: Protein ID: {protein_id}, Domain Info: {domain_info}")

        # Regular expression to find the LOV domain tuple
        match = re.search(r'LOV : \((\d+), (\d+)\)', domain_info)
        if match:
            start, end = match.groups()
            lov_domain_data.append((protein_id, (int(start), int(end))))

    return lov_domain_data

In [12]:
lov_data = extract_lov_domain_tuples(XLS_msa)
# print(lov_data)

Debug: Protein ID: A0A009EVU7, Domain Info: [N_terminus : (0, 0) ---> GAF_2 : (9, 166) ---> PAS_4 : (186, 293) ---> LOV : (309, 414) ---> GGDEF : (426, 583) ---> EAL : (601, 839) ---> C_terminus : (850, 850)]
Debug: Protein ID: A0A009JQP1, Domain Info: [N_terminus : (0, 0) ---> GAF_2 : (26, 166) ---> PAS_4 : (187, 293) ---> LOV : (309, 414) ---> GGDEF : (426, 583) ---> EAL : (604, 840) ---> C_terminus : (855, 855)]
Debug: Protein ID: A0A009NCY2, Domain Info: [N_terminus : (0, 0) ---> GAF_2 : (26, 166) ---> PAS_4 : (187, 293) ---> LOV : (309, 414) ---> GGDEF : (426, 583) ---> EAL : (604, 840) ---> C_terminus : (855, 855)]
Debug: Protein ID: A0A010FRR9, Domain Info: [N_terminus : (0, 0) ---> GAF_2 : (26, 166) ---> PAS_4 : (187, 293) ---> LOV : (309, 414) ---> GGDEF : (426, 583) ---> EAL : (604, 840) ---> C_terminus : (855, 855)]
Debug: Protein ID: A0A010PA15, Domain Info: [N_terminus : (0, 0) ---> LOV : (450, 555) ---> HisKA : (576, 641) ---> HATPase_c : (688, 803) ---> Response_reg : (8